In [218]:
#!pip install tensorflow pandas sklearn numpy

## Goals

 - Loading the frames generated by our `simulator.py`
     - TODO: Generate Realtime System Simulations with different behaviours (No, Medium or Strong Perturbations)
     - TODO: Label the frames correctly
 - Preprocess the frames
 - Select a first model for experimentation purposes (TensorFlow)
 - Train the model on the loaded frames
 - Analyze the model performance on newly generated frames
 - Adjust either the model or the simulator and repeat all the steps above.

In [219]:
import tensorflow as tf

In [220]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [221]:
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib

In [222]:
tf.__version__

'2.1.0'

In [223]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = os.path.join(".", 'frames', 'train')
validation_dir = os.path.join(".", 'frames', 'valid')

In [224]:
train_stable_dir = os.path.join(train_dir, 'stable')
train_malfunction_dir = os.path.join(train_dir, 'malfunction')

validation_stable_dir = os.path.join(validation_dir, 'stable')
validation_malfunction_dir = os.path.join(validation_dir, 'malfunction')

In [225]:
train_stable_num = len(os.listdir(train_stable_dir))
train_malfunction_num = len(os.listdir(train_malfunction_dir))

validation_stable_num = len(os.listdir(validation_stable_dir))
validation_malfunction_num = len(os.listdir(validation_malfunction_dir))

total_train = train_stable_num + train_malfunction_num
total_validation = train_stable_num + train_malfunction_num

In [226]:
print('total training stable images:', train_stable_num)
print('total training malfunction images:', train_malfunction_num)

print('total validation stable images:', validation_stable_num)
print('total validation malfunction images:', validation_malfunction_num)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_validation)

total training stable images: 500
total training malfunction images: 500
--
Total training images: 1000


In [227]:
batch_size = 128
epochs = 15
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [236]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [237]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')
validation_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=validation_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [230]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

In [231]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [232]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 224, 224, 16)      448       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 112, 112, 32)      4640      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 56, 56, 64)        18496     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 28, 28, 64)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 50176)            

In [233]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=validation_data_gen,
    validation_steps=total_validation // batch_size
)

  ...
    to  
  ['...']
Train for 7 steps
Epoch 1/15
7/7 [==============================] - 33s 5s/step - loss: 3.7379 - accuracy: 0.4839
Epoch 2/15
7/7 [==============================] - 29s 4s/step - loss: 0.6865 - accuracy: 0.5011
Epoch 3/15
7/7 [==============================] - 29s 4s/step - loss: 0.5964 - accuracy: 0.5011
Epoch 4/15
7/7 [==============================] - 28s 4s/step - loss: 0.4113 - accuracy: 0.7248
Epoch 5/15
7/7 [==============================] - 30s 4s/step - loss: 0.2756 - accuracy: 0.8956
Epoch 6/15
7/7 [==============================] - 35s 5s/step - loss: 0.2070 - accuracy: 0.9186
Epoch 7/15
7/7 [==============================] - 31s 4s/step - loss: 0.1584 - accuracy: 0.9438
Epoch 8/15
7/7 [==============================] - 29s 4s/step - loss: 0.1594 - accuracy: 0.9289
Epoch 9/15
7/7 [==============================] - 29s 4s/step - loss: 0.1542 - accuracy: 0.9392
Epoch 10/15
7/7 [==============================] - 30s 4s/step - loss: 0.1520 - accuracy: 0.9

In [245]:
model.evaluate_generator(validation_data_gen)

  ...
    to  
  ['...']


[0.11043732799589634, 0.96]